In [38]:
import pandas as pd
import matplotlib.pyplot as plt
from General_Functions import long_horizon_ret
from Main import settings, features, pf_set
from pandas.tseries.offsets import MonthEnd
from Prepare_Data import process_risk_free_rate, process_return_data, wealth_func, load_and_filter_market_returns_test, process_all_data, process_cluster_labels
from General_Functions import size_screen_fun, addition_deletion_fun
from Estimate_Covariance_Matrix import process_cluster_labels
import numpy as np
from scipy.stats import rankdata
import re
import gc

In [39]:
#Indhenter chars og daily:
file_path_usa_test = "./data_test/usa_test.parquet"
daily_file_path = "./data_test/usa_dsf_test.parquet"
file_path_world_ret = "./data_test/world_ret_test.csv"
risk_free_path = "./data_test/risk_free_test.csv"
market_path = "./data_test/market_returns_test.csv"

# Kald funktionen
chars, daily = process_all_data(file_path_usa_test, daily_file_path, file_path_world_ret, risk_free_path, market_path)
print(chars)
print(daily)

All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%
          id        eom   sic size_grp            me  rvol_252d  dolvol_126d  \
1      10104 2010-02-28  7372     mega  123619.74809   0.343023     0.854651   
2      10104 2010-03-31  7372     mega  129040.82501   0.267442     0.854651   
3      10104 2010-04-30  7372     mega  130101.18774   0.284884     0.848837   
4      10104 2010-05-31  7372     mega  113436.81847   0.354651     0.848837   
5      10104 2010-06-30  7372     mega  107857.95540   0.261628     0.843023   
...      ...        ...   ...      ...           ...        ...          ...   
28707  92655 2023-06-30  6324     mega  447491.23412   0.284884     0.953488   
28708  92655 2023-07-31  6324     mega  471446.66929   0.319767     0.953488   
28709  92655 2023-08-31  6324     mega  441458.42446   0.348837     0.941860   
28710  926

## Prepare cluster characteristics ---------------

In [40]:
file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"
cluster_labels = process_cluster_labels(file_path_cluster_labels, file_path_factor_details)
print(cluster_labels)

       characteristic       cluster  direction
0                 age  low_leverage       -1.0
1             aliq_at    investment       -1.0
2            aliq_mat  low_leverage       -1.0
3            ami_126d          size        1.0
4               at_be  low_leverage       -1.0
..                ...           ...        ...
149           z_score  low_leverage        1.0
150  zero_trades_126d      low_risk        1.0
151   zero_trades_21d      low_risk        1.0
152  zero_trades_252d      low_risk        1.0
153         rvol_252d      low_risk       -1.0

[154 rows x 3 columns]


In [41]:
# Prepare cluster characteristics
valid_chars = chars[chars["valid"] == True]
cluster_data_m = valid_chars[["id", "eom", "size_grp", "ff12"] + features].copy()

clusters = cluster_labels["cluster"].unique()
cluster_ranks = {}

for cl in clusters:
    chars_sub = cluster_labels[(cluster_labels["cluster"] == cl) & (cluster_labels["characteristic"].isin(features))]

    data_sub = cluster_data_m[chars_sub["characteristic"].values].copy()

    for c in chars_sub["characteristic"].values:
        dir_value = chars_sub.loc[chars_sub["characteristic"] == c, "direction"].values[0]
        if dir_value == -1:
            data_sub[c] = 1 - data_sub[c]

    cluster_ranks[cl] = data_sub.mean(axis=1)

# Konverter cluster ranks til en dataframe
cluster_ranks_df = pd.DataFrame(cluster_ranks)

# Kombiner med de oprindelige data
cluster_data_m = cluster_data_m[["id", "eom", "size_grp", "ff12"]].copy()
#cluster_data_m["eom_ret"] = cluster_data_m["eom"] + pd.DateOffset(months=1) - pd.DateOffset(days=1) #Ikke helt korrekt
cluster_data_m["eom_ret"] = cluster_data_m["eom"] + pd.DateOffset(months=1)
cluster_data_m["eom_ret"] = cluster_data_m["eom_ret"] + pd.offsets.MonthEnd(0)  # Sikrer, at det er den sidste dag i måneden



# Merge cluster rankings
cluster_data_m = pd.concat([cluster_data_m, cluster_ranks_df], axis=1)

# Output result
print(cluster_data_m)

          id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
24     10104 2012-01-31     mega  BusEq 2012-02-29      0.763518    0.497303   
25     10104 2012-02-29     mega  BusEq 2012-03-31      0.775977    0.498864   
26     10104 2012-03-31     mega  BusEq 2012-04-30      0.772860    0.593682   
27     10104 2012-04-30     mega  BusEq 2012-05-31      0.723207    0.556345   
28     10104 2012-05-31     mega  BusEq 2012-06-30      0.742939    0.566149   
...      ...        ...      ...    ...        ...           ...         ...   
28707  92655 2023-06-30     mega  Money 2023-07-31      0.525670    0.231943   
28708  92655 2023-07-31     mega  Money 2023-08-31      0.469914    0.189911   
28709  92655 2023-08-31     mega  Money 2023-09-30      0.519169    0.187129   
28710  92655 2023-09-30     mega  Money 2023-10-31      0.506901    0.200052   
28711  92655 2023-10-31     mega  Money 2023-11-30      0.511762    0.223770   

           size     value   quality  lo

In [42]:
# Add Industry/market dummies
if settings["cov_set"]["industries"]:
    # Opret industry dummies
    industries = sorted(cluster_data_m["ff12"].unique())
    for ind in industries:
        cluster_data_m[str(ind)] = (cluster_data_m["ff12"] == ind).astype(int)
    ind_factors = industries
else:
    # Tilføj markedsfaktor
    cluster_data_m["mkt"] = 1
    ind_factors = ["mkt"]

# Output test
print(cluster_data_m)

          id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
24     10104 2012-01-31     mega  BusEq 2012-02-29      0.763518    0.497303   
25     10104 2012-02-29     mega  BusEq 2012-03-31      0.775977    0.498864   
26     10104 2012-03-31     mega  BusEq 2012-04-30      0.772860    0.593682   
27     10104 2012-04-30     mega  BusEq 2012-05-31      0.723207    0.556345   
28     10104 2012-05-31     mega  BusEq 2012-06-30      0.742939    0.566149   
...      ...        ...      ...    ...        ...           ...         ...   
28707  92655 2023-06-30     mega  Money 2023-07-31      0.525670    0.231943   
28708  92655 2023-07-31     mega  Money 2023-08-31      0.469914    0.189911   
28709  92655 2023-08-31     mega  Money 2023-09-30      0.519169    0.187129   
28710  92655 2023-09-30     mega  Money 2023-10-31      0.506901    0.200052   
28711  92655 2023-10-31     mega  Money 2023-11-30      0.511762    0.223770   

           size     value   quality  ..

In [43]:
# Standardiser faktorer (clusters) per eom
cluster_data_m[clusters] = cluster_data_m.groupby("eom")[clusters].transform(lambda x: (x - x.mean()) / x.std())
print(cluster_data_m)

          id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
24     10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
25     10104 2012-02-29     mega  BusEq 2012-03-31      1.773561   -0.005543   
26     10104 2012-03-31     mega  BusEq 2012-04-30      1.774189    0.553260   
27     10104 2012-04-30     mega  BusEq 2012-05-31      1.422269    0.312431   
28     10104 2012-05-31     mega  BusEq 2012-06-30      1.560002    0.372443   
...      ...        ...      ...    ...        ...           ...         ...   
28707  92655 2023-06-30     mega  Money 2023-07-31      0.192269   -1.748730   
28708  92655 2023-07-31     mega  Money 2023-08-31     -0.216928   -2.017339   
28709  92655 2023-08-31     mega  Money 2023-09-30      0.147315   -2.030984   
28710  92655 2023-09-30     mega  Money 2023-10-31      0.052639   -1.975682   
28711  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   

           size     value   quality  ..

In [44]:
unique_dates = daily["eom"].unique()
print(unique_dates[20:50])

<DatetimeArray>
['2011-10-31 00:00:00', '2011-11-30 00:00:00', '2011-12-31 00:00:00',
 '2012-01-31 00:00:00', '2012-02-29 00:00:00', '2012-03-31 00:00:00',
 '2012-04-30 00:00:00', '2012-05-31 00:00:00', '2012-06-30 00:00:00',
 '2012-07-31 00:00:00', '2012-08-31 00:00:00', '2012-09-30 00:00:00',
 '2012-10-31 00:00:00', '2012-11-30 00:00:00', '2012-12-31 00:00:00',
 '2013-01-31 00:00:00', '2013-02-28 00:00:00', '2013-03-31 00:00:00',
 '2013-04-30 00:00:00', '2013-05-31 00:00:00', '2013-06-30 00:00:00',
 '2013-07-31 00:00:00', '2013-08-31 00:00:00', '2013-09-30 00:00:00',
 '2013-10-31 00:00:00', '2013-11-30 00:00:00', '2013-12-31 00:00:00',
 '2014-01-31 00:00:00', '2014-02-28 00:00:00', '2014-03-31 00:00:00']
Length: 30, dtype: datetime64[ns]


In [45]:
# Tilføj daglige returdata
min_eom = cluster_data_m["eom"].min()
daily_filtered = daily[daily["date"] >= min_eom][["id", "date", "ret_exc", "eom"]].copy()
daily_filtered.rename(columns={"eom": "eom_ret"}, inplace=True)

# Flet daglige data med cluster_data_m
cluster_data_d = cluster_data_m.merge(daily_filtered, on=["id", "eom_ret"], how="inner")

print(cluster_data_d)
print(cluster_data_d.shape)
# Fjern rækker med manglende værdier
cluster_data_d.dropna(inplace=True)

print(cluster_data_d)
print(cluster_data_d.shape)

           id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
0       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
1       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
2       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
3       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
4       10104 2012-01-31     mega  BusEq 2012-02-29      1.680821   -0.014307   
...       ...        ...      ...    ...        ...           ...         ...   
511495  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   
511496  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   
511497  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   
511498  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   
511499  92655 2023-10-31     mega  Money 2023-11-30      0.091804   -1.820302   

            size     value 

#### Now in a function

In [46]:
def process_cluster_data(chars,daily, cluster_labels_path, factor_details_path):
    """
    Behandler cluster data ved at læse og forberede cluster labels, beregne cluster ranks,
    tilføje industry/market dummies, standardisere faktorer og flette daglige returdata.

    Parametre:
    - chars: DataFrame med karakteristika for aktier
    - daily: DataFrame med daglige afkastdata
    - cluster_labels_path: Sti til CSV-filen med cluster labels
    - factor_details_path: Sti til Excel-filen med factor details

    Returnerer:
    - cluster_data_d: Behandlet DataFrame med alle transformationer

    Eksempel på brug:
    file_path_cluster_labels = "Data/Cluster Labels.csv"
    file_path_factor_details = "Data/Factor Details.xlsx"
    cluster_data_d = process_cluster_data(chars, daily, file_path_cluster_labels, file_path_factor_details)
    
    """

    # Læs cluster labels
    cluster_labels = pd.read_csv(cluster_labels_path)
    cluster_labels["cluster"] = cluster_labels["cluster"].str.lower().str.replace(r"\s|-", "_", regex=True)

    # Læs factor details
    factor_signs = pd.read_excel(factor_details_path, usecols=["abr_jkp", "direction"])
    factor_signs.rename(columns={"abr_jkp": "characteristic"}, inplace=True)
    factor_signs.dropna(subset=["characteristic"], inplace=True)
    factor_signs["direction"] = factor_signs["direction"].astype(float)

    # Merge factor signs med cluster labels
    cluster_labels = cluster_labels.merge(factor_signs, on="characteristic", how="left")

    # Tilføj specifikt cluster til rvol_252d
    new_row = pd.DataFrame({"characteristic": ["rvol_252d"], "cluster": ["low_risk"], "direction": [-1]})
    cluster_labels = pd.concat([cluster_labels, new_row], ignore_index=True)

    # Filtrer gyldige karakteristika
    valid_chars = chars[chars["valid"] == True]
    cluster_data_m = valid_chars[["id", "eom", "size_grp", "ff12"] + features].copy()

    # Beregn cluster ranks
    clusters = cluster_labels["cluster"].unique()
    cluster_ranks = {}

    for cl in clusters:
        chars_sub = cluster_labels[(cluster_labels["cluster"] == cl) & (cluster_labels["characteristic"].isin(features))]
        data_sub = cluster_data_m[chars_sub["characteristic"].values].copy()

        for c in chars_sub["characteristic"].values:
            dir_value = chars_sub.loc[chars_sub["characteristic"] == c, "direction"].values[0]
            if dir_value == -1:
                data_sub[c] = 1 - data_sub[c]

        cluster_ranks[cl] = data_sub.mean(axis=1)

    # Konverter cluster ranks til DataFrame
    cluster_ranks_df = pd.DataFrame(cluster_ranks)

    # Kombiner med oprindelige data
    cluster_data_m = cluster_data_m[["id", "eom", "size_grp", "ff12"]].copy()
    cluster_data_m["eom_ret"] = cluster_data_m["eom"] + pd.DateOffset(months=1)
    cluster_data_m["eom_ret"] = cluster_data_m["eom_ret"] + pd.offsets.MonthEnd(0)

    # Merge cluster rankings
    cluster_data_m = pd.concat([cluster_data_m, cluster_ranks_df], axis=1)

    # Tilføj industry/market dummies
    if settings["cov_set"]["industries"]:
        industries = sorted(cluster_data_m["ff12"].unique())
        for ind in industries:
            cluster_data_m[str(ind)] = (cluster_data_m["ff12"] == ind).astype(int)
        ind_factors = industries
    else:
        cluster_data_m["mkt"] = 1
        ind_factors = ["mkt"]

    # Standardiser faktorer per eom
    cluster_data_m[clusters] = cluster_data_m.groupby("eom")[clusters].transform(lambda x: (x - x.mean()) / x.std(ddof=0))
    cluster_data_m[clusters] = cluster_data_m[clusters].fillna(0)

    # Tilføj daglige returdata
    min_eom = cluster_data_m["eom"].min()
    daily_filtered = daily[daily["date"] >= min_eom][["id", "date", "ret_exc", "eom"]].copy()
    daily_filtered.rename(columns={"eom": "eom_ret"}, inplace=True)

    # Flet daglige data med cluster_data_m
    cluster_data_d = cluster_data_m.merge(daily_filtered, on=["id", "eom_ret"], how="inner")

    # Fjern rækker med manglende værdier
    cluster_data_d.dropna(inplace=True)

    return cluster_data_d


In [47]:
file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"
cluster_data_d = process_cluster_data(chars, daily, file_path_cluster_labels, file_path_factor_details)
print(cluster_data_d)
print(cluster_data_d.shape)

           id        eom size_grp   ff12    eom_ret  low_leverage  investment  \
0       10104 2012-01-31     mega  BusEq 2012-02-29      1.685728   -0.014349   
1       10104 2012-01-31     mega  BusEq 2012-02-29      1.685728   -0.014349   
2       10104 2012-01-31     mega  BusEq 2012-02-29      1.685728   -0.014349   
3       10104 2012-01-31     mega  BusEq 2012-02-29      1.685728   -0.014349   
4       10104 2012-01-31     mega  BusEq 2012-02-29      1.685728   -0.014349   
...       ...        ...      ...    ...        ...           ...         ...   
511495  92655 2023-10-31     mega  Money 2023-11-30      0.092072   -1.825616   
511496  92655 2023-10-31     mega  Money 2023-11-30      0.092072   -1.825616   
511497  92655 2023-10-31     mega  Money 2023-11-30      0.092072   -1.825616   
511498  92655 2023-10-31     mega  Money 2023-11-30      0.092072   -1.825616   
511499  92655 2023-10-31     mega  Money 2023-11-30      0.092072   -1.825616   

            size     value 

### Importeret funktion

In [ ]:


file_path_cluster_labels = "Data/Cluster Labels.csv"
file_path_factor_details = "Data/Factor Details.xlsx"
cluster_data_d = process_cluster_data(chars, daily, file_path_cluster_labels, file_path_factor_details)

## Create factor returns --------------------